In [32]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch

import torch.nn as nn
import torch
from torchvision import models
from utils import save_net,load_net

%matplotlib inline

In [33]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [34]:
root = 'C:/Users/Natalie/Documents/GitHub/cnn-model/data/SHT'

In [35]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [36]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [37]:
img_paths

['C:/Users/Natalie/Documents/GitHub/cnn-model/data/SHT\\part_A_final/test_data\\images\\IMG_1.jpg',
 'C:/Users/Natalie/Documents/GitHub/cnn-model/data/SHT\\part_A_final/test_data\\images\\IMG_2.jpg',
 'C:/Users/Natalie/Documents/GitHub/cnn-model/data/SHT\\part_A_final/test_data\\images\\IMG_3.jpg']

In [38]:
class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
            for i in range(len(self.frontend.state_dict().items())):
                list(self.frontend.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i][1].data[:]
    def forward(self,x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [39]:
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

In [40]:
model = CSRNet()

In [41]:
python train.py part_A_train.json part_A_val.json 0 0

SyntaxError: invalid syntax (<ipython-input-41-2f73f20d27a1>, line 1)

In [31]:
checkpoint = torch.load('model_best.pth.tar')

FileNotFoundError: [Errno 2] No such file or directory: 'model_best.pth.tar'

In [39]:
model.load_state_dict(checkpoint['state_dict'])

In [45]:
mae = 0
for i in xrange(len(img_paths)):
    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB'))

    img[0,:,:]=img[0,:,:]-92.8207477031
    img[1,:,:]=img[1,:,:]-95.2757037428
    img[2,:,:]=img[2,:,:]-104.877445883
    img = img.cuda()
    #img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print i,mae
print mae/len(img_paths)

0 15.50390625
1 60.9075317383
2 220.511169434
3 239.312469482
4 252.252349854
5 272.965286255
6 457.101577759
7 651.92250061
8 681.363113403
9 785.472061157
10 838.996322632
11 1012.11277771
12 1073.18791199
13 1074.72886658
14 1139.53701782
15 1201.55630493
16 1316.97366333
17 1447.83328247
18 1578.5967865
19 1622.11299896
20 1650.6510849
21 1756.23152924
22 1810.32888031
23 1816.16628265
24 1864.3579483
25 1886.67713928
26 1942.00212097
27 1995.17939758
28 2031.85020447
29 2236.5879364
30 2265.58026123
31 2272.81892395
32 2333.38215637
33 2489.37367249
34 2560.18891907
35 2580.02906799
36 2588.45735168
37 2661.36177063
38 2788.47312927
39 2900.07542419
40 2900.83190918
41 2976.19485474
42 2995.64665985
43 3058.02416229
44 3103.91133881
45 3241.52921295
46 3906.06101227
47 3918.4709549
48 3948.23314667
49 3953.28383636
50 3989.9439621
51 4093.59087372
52 4180.21788788
53 4289.41963959
54 4291.79859161
55 4302.33109283
56 4339.80475616
57 4543.3482132
58 4626.09952545
59 4684.85929108
